# Meraki Python SDK Demo: Offline Switch Finder

*This notebook demonstrates using the Meraki Python SDK to create a list of all the switches in a network that are offline.*

---

>NB: Throughout this notebook, we will print values for demonstration purposes. In a production Python script, the coder would likely remove these print statements to clean up the console output.

In this first cell, we import the required meraki and os modules, and open the Dashboard API connection using the SDK.

In [1]:
# Rely on meraki SDK, os, and tablib -- more on tablib later
import meraki
import os
import tablib

# We're also going to import Python's built-in JSON module, but only to make the console output pretty. In production, you wouldn't need any of the printing calls at all, nor this import!
import json

# Setting API key this way, and storing it in the env variables, lets us keep the sensitive API key out of the script itself
# The meraki.DashboardAPI() method does not require explicitly passing this value; it will check the environment for a variable
# called 'MERAKI_DASHBOARD_API_KEY' on its own. In this case, API_KEY is shown simply as an reference to where that information is
# stored.
API_KEY = os.getenv('MERAKI_DASHBOARD_API_KEY')

# Initialize the Dashboard connection.
dashboard = meraki.DashboardAPI()

meraki:     INFO > Meraki dashboard API session initialized with these parameters: {'version': '1.0.0b13', 'api_key': '************************************dfb9', 'base_url': 'https://api.meraki.com/api/v1', 'single_request_timeout': 60, 'certificate_path': '', 'requests_proxy': '', 'wait_on_rate_limit': True, 'nginx_429_retry_wait_time': 60, 'action_batch_retry_wait_time': 60, 'retry_4xx_error': False, 'retry_4xx_error_wait_time': 60, 'maximum_retries': 2, 'simulate': False, 'be_geo_id': None, 'caller': None}


Let's make a basic pretty print formatter, `printj()`. It will make reading the JSON later a lot easier, but won't be necessary in production scripts.

In [2]:
def printj(ugly_json_object):
    
    # The json.dumps() method converts a JSON object into human-friendly formatted text
    pretty_json_string = json.dumps(ugly_json_object, indent = 2, sort_keys = False)

    return print(pretty_json_string)

Most API calls require passing values for the organization ID and/or the network ID. In this second cell, we fetch a list of the organizations the API key can access, then pick the first org in the list, and the first network in that organization, to use for later operations. You could re-use this code presuming your API key only has access to a single organization, and that organization only contains a single network. Otherwise, you would want to review the organizations object declared and printed here to review its contents.

In [3]:
# Let's make it easier to call this data later
# getOrganizations will return all orgs to which the supplied API key has access
organizations = dashboard.organizations.getOrganizations()
print('Organizations:')
printj(organizations)

# This example presumes we want to use the first organization as the scope for later operations. 
firstOrganizationId = organizations[0]['id']
firstOrganizationName = organizations[0]['name']

# Print a blank line for legibility before showing the firstOrganizationId
print('')
print(f'The firstOrganizationId is {firstOrganizationId}, and its name is {firstOrganizationName}.')

meraki:     INFO > organizations, getOrganizations - 200 OK
Organizations:
[
  {
    "id": "102908",
    "name": "Olympus",
    "url": "https://n80.meraki.com/o/99uSJa/manage/organization/overview"
  }
]

The firstOrganizationId is 102908, and its name is Olympus.


This example presumes we want to use the first network of the chosen organization as the scope for later operations. It is fine to re-use presuming that your organization only contains a single network. Otherwise, you would want to review the organizations object declared and printed below to review its contents.

In [4]:
networks = dashboard.organizations.getOrganizationNetworks(organizationId=firstOrganizationId)
print('Networks:')
printj(networks)

firstNetworkId = networks[0]['id']
firstNetworkName = networks[0]['name']

# Print a blank line for legibility before showing the firstNetworkId
print('')

print(f'The firstNetworkId is {firstNetworkId}, and its name is {firstNetworkName}.')

meraki:     INFO > organizations, getOrganizationNetworks; page 1 - 200 OK
Networks:
[
  {
    "enrollmentString": null,
    "id": "L_607985949695028290",
    "name": "Edgehill2",
    "organizationId": "102908",
    "productTypes": [
      "appliance",
      "camera",
      "cellularGateway",
      "switch",
      "wireless"
    ],
    "tags": [],
    "timeZone": "America/Los_Angeles",
    "url": "https://n80.meraki.com/Edgehill2-applia/n/wyTiUcqb/manage/usage/list"
  },
  {
    "enrollmentString": null,
    "id": "L_607985949695028346",
    "name": "Virtual",
    "organizationId": "102908",
    "productTypes": [
      "appliance",
      "camera",
      "switch",
      "wireless"
    ],
    "tags": [],
    "timeZone": "America/Los_Angeles",
    "url": "https://n80.meraki.com/Virtual-camera/n/mlMVmcqb/manage/usage/list"
  },
  {
    "enrollmentString": null,
    "id": "L_607985949695028380",
    "name": "Western",
    "organizationId": "102908",
    "productTypes": [
      "appliance",


Now that we've got the organization and network values figured out, we can get to the ask at hand:

> Show me a list of all the offline switches in a network.

The `getOrganizationDevicesStatuses` endpoint will return the devices (switches and otherwise) and their statuses, but it will not return their model numbers. To get that info, we use `getOrganizationDevices`. 

So first, we create the `devices` list. Then we create a list of those devices' statuses in `device_statuses`. Then, we use a [list comprehension](https://www.datacamp.com/community/tutorials/python-list-comprehension) to find all instances of switches in the `devices` list and put them in a new list, `devices_switches`.  

In [5]:
devices = dashboard.organizations.getOrganizationDevices(organizationId=firstOrganizationId)
devices_statuses = dashboard.organizations.getOrganizationDevicesStatuses(organizationId=firstOrganizationId)

# Let's get the switch devices list
devices_switches = [i for i in devices if 'MS' in i['model']]
print('These are the switches:')
printj(devices_switches)

# Make a new list of all the serials from devices_switches
devices_switches_serials = [i['serial'] for i in devices_switches]

meraki:     INFO > organizations, getOrganizationDevices; page 1 - 200 OK
      meraki:     INFO > organizations, getOrganizationDevicesStatuses; page 1 - 200 OK
These are the switches:
[
  {
    "address": "4111 Edgehill Dr, Los Angeles, CA 90008",
    "configurationUpdatedAt": "2020-07-21T01:57:44Z",
    "firmware": "switch-12-17",
    "lanIp": "172.17.0.4",
    "lat": 34.00968,
    "lng": -118.33069,
    "mac": "34:56:fe:cc:b9:84",
    "model": "MS120-8LP",
    "name": "Edgehill MS120-8LP",
    "networkId": "L_607985949695028290",
    "notes": "",
    "serial": "Q2BX-DEZ3-LGMD",
    "tags": [],
    "url": "https://n80.meraki.com/Edgehill2-switch/n/V3nOZcqb/manage/nodes/new_list/57548246661508"
  },
  {
    "address": "",
    "configurationUpdatedAt": "2020-07-07T00:08:05Z",
    "firmware": "Not running configured version",
    "lanIp": null,
    "lat": 37.4180951010362,
    "lng": -122.098531723022,
    "mac": "4c:c8:a1:06:00:f8",
    "model": "MS250-24",
    "name": "",
    "networ

Great! Now we have a list of all the switches and their statuses `devices_switches`, as well a separate list of just their serials `devices_switches_serials`, but we only want the ones that are offline. So here, we'll use more list comprehensions to narrow down the list and create a new list with only the information we need.

In [6]:
# Now we can list out the statuses (and whatever metadata we need) for these switches
print('We can list out the statuses (and whatever metadata we need) for these switches:')
devices_statuses_switches = [{'serial': i['serial'], 'status':i['status']} for i in devices_statuses if i['serial'] in devices_switches_serials]
print('Switch statuses are:')
printj(devices_statuses_switches)

# Print a blank line for legibility
print('')

# We can narrow it down to the ones that are offline
print('We can narrow it down to the ones that are offline:')
devices_statuses_switches_offline = [i for i in devices_statuses_switches if i['status'] != 'online']
print('Offline switches are:')
printj(devices_statuses_switches_offline)

We can list out the statuses (and whatever metadata we need) for these switches:
Switch statuses are:
[
  {
    "serial": "Q2BX-DEZ3-LGMD",
    "status": "offline"
  },
  {
    "serial": "QBSB-AK7F-URKC",
    "status": "offline"
  },
  {
    "serial": "QBSB-BDX9-8AZP",
    "status": "offline"
  },
  {
    "serial": "QBSB-DD84-RCQS",
    "status": "offline"
  },
  {
    "serial": "QBSB-DLV3-235C",
    "status": "offline"
  },
  {
    "serial": "QBSB-E3LG-VEQZ",
    "status": "offline"
  },
  {
    "serial": "QBSB-HXH8-6N32",
    "status": "offline"
  },
  {
    "serial": "QBSB-PP5Z-HJ84",
    "status": "offline"
  },
  {
    "serial": "QBSB-YYWS-RCY7",
    "status": "offline"
  }
]

We can narrow it down to the ones that are offline:
Offline switches are:
[
  {
    "serial": "Q2BX-DEZ3-LGMD",
    "status": "offline"
  },
  {
    "serial": "QBSB-AK7F-URKC",
    "status": "offline"
  },
  {
    "serial": "QBSB-BDX9-8AZP",
    "status": "offline"
  },
  {
    "serial": "QBSB-DD84-RCQS",
  

We started from the devices list, and the devices statuses list, and created a list of offline switches. Now let's look at when they last reported to the Dashboard.

In [7]:
# Another list comprehension!
devices_last_reported_times = [{'serial': i['serial'], 'lastReportedAt': i['lastReportedAt']} for i in devices_statuses if i['serial'] in devices_switches_serials]

printj(devices_last_reported_times)

[
  {
    "lastReportedAt": "2020-07-22 20:14:17.501",
    "serial": "Q2BX-DEZ3-LGMD"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-AK7F-URKC"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-BDX9-8AZP"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-DD84-RCQS"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-DLV3-235C"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-E3LG-VEQZ"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-HXH8-6N32"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-PP5Z-HJ84"
  },
  {
    "lastReportedAt": null,
    "serial": "QBSB-YYWS-RCY7"
  }
]


# Final steps

Excellent, now we have a list of offline switches, and it's pretty easy to read. But what if we could use this in an Excel format? Well, there's a Python module for that, too. In this case, we're using [tablib](https://pypi.org/project/tablib/).

In [13]:
# Let's convert that JSON-formatted data to a tabular dataset. You can copy/paste this into Excel, or write additional Python to create a new Excel file entirely!
excel_formatted = tablib.import_set(json.dumps(devices_last_reported_times), format = 'json')

# Let's see how it looks!
print(excel_formatted)

serial        |lastReportedAt         
--------------|-----------------------
Q2BX-DEZ3-LGMD|2020-07-22 20:14:17.501
QBSB-AK7F-URKC|None                   
QBSB-BDX9-8AZP|None                   
QBSB-DD84-RCQS|None                   
QBSB-DLV3-235C|None                   
QBSB-E3LG-VEQZ|None                   
QBSB-HXH8-6N32|None                   
QBSB-PP5Z-HJ84|None                   
QBSB-YYWS-RCY7|None                   


# Final thoughts

And we're done! Hopefully you found this a useful demonstration of just a few things that are possible with Meraki's Python SDK. These additional resources may prove useful along the way.

[Meraki Interactive API Docs](https://developer.cisco.com/meraki/api-v1/#!overview): The official (and interactive!) Meraki API and SDK documentation repository on DevNet.

[VS Code](https://code.visualstudio.com/): An excellent code editor with full support for Python and Python notebooks.

[Automate the Boring Stuff with Python](https://automatetheboringstuff.com/): An excellent learning resource that puts the real-world problem first, then teaches you the Pythonic solution along the way.